In [1]:
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'code')))

import torch
from skip_block import SkipBlock
import torch.nn.functional as F
from typing import Optional


EPS = 1e-8
DEBUG = True
INIT_COEF = 4
LR = 0.7
DECAY = 0.9
DECAY_STEP = 1
EPOCHS = 20


def eliding_w(input_dim: int, target: int) -> torch.Tensor:
    w = torch.zeros(input_dim, input_dim)
    for i in range(input_dim):
        w[i][i] += 1
        w[i][target] -= 1
    return w


def skip_application(skip_dim: int) -> torch.Tensor:
    w = torch.zeros(skip_dim, 2*skip_dim)
    w[:, :skip_dim] = w[:, skip_dim:skip_dim*2] = torch.eye(skip_dim)
    return w


def dp_verify(model: torch.nn.Module, input: torch.Tensor, eps: float, true_output: int) -> bool:
    input = input.unsqueeze(0)
    shallowuni = Deeppoly(input-eps, input+eps)
    shallowuni.forward(model)
    if shallowuni.append_and_verify(true_output):
        return True
    if shallowuni.alpha_buffer_idx == 0:
        return False
    for i in range(shallowuni.alpha_buffer_idx):
        shallowuni.alpha_buffer[i].requires_grad = True

    if DEBUG:
        torch.autograd.set_detect_anomaly(True)

    optimizer = torch.optim.Adam(shallowuni.alpha_buffer, lr=LR)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=DECAY_STEP, gamma=DECAY)
    for epoch in range(EPOCHS):
        optimizer.zero_grad()
        shallowuni.rewind()
        shallowuni.forward(model)
        if shallowuni.append_and_verify(true_output):
            return True
        loss = torch.mean(torch.relu(shallowuni.upper_bounds[-1]).flatten())
        loss.backward()
        optimizer.step()
        print(sum(shallowuni.alpha_buffer[-1].grad[0]))
        if scheduler.get_last_lr()[0] > 0.1:
            scheduler.step()
    return False
    


def backsubstitute(W_l0: torch.Tensor, W_u0: torch.Tensor, W_l: torch.Tensor, W_u: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
    W_l_pos = W_l.clamp_min(0)
    W_l_neg = W_l.clamp_max(0)
    W_u_pos = W_u.clamp_min(0)
    W_u_neg = W_u.clamp_max(0)

    W_l_tilde = W_l0 @ W_l_pos + W_u0 @ W_l_neg
    W_u_tilde = W_u0 @ W_u_pos + W_l0 @ W_u_neg
    return W_l_tilde, W_u_tilde


class Deeppoly():
    def __init__(self, init_lower_bound: torch.Tensor, init_upper_bound: torch.Tensor):
        # numerical boundaries
        self.lower_bounds: list[torch.Tensor] = [init_lower_bound]
        self.upper_bounds: list[torch.Tensor] = [init_upper_bound]
        # the lock mechanism will be buggy if there's skip in skip, which is not the case for our nets
        # the lock mechanism is to ensure that the linear layer absorbed does not cause problem for skip
        self.lock_idx : Optional[int] = None
        # linear constraint expressions can be stored as matrices multipliers (x' \leq Ax + b)
        # x' <> x (A| 0)
        #         (b| 1)
        self.lower_const_weights: list[torch.Tensor] = []
        self.upper_const_weights: list[torch.Tensor] = []

        self.alpha_buffer: list[torch.nn.Parameter] = []
        self.alpha_buffer_idx: int = 0

        self.forward_net = {
            torch.nn.Sequential: self._sequential,
            SkipBlock: self._skip,
            torch.nn.ReLU6: self._relu6,
        }

        self.forward_layer = {
            torch.nn.Conv2d: self._conv2d,
            torch.nn.Linear: self._linear,
            torch.nn.ReLU: self._relu,
            torch.nn.Flatten: self._flatten,
        }


    def rewind(self):
        self.lower_bounds = self.lower_bounds[:1]
        self.upper_bounds = self.upper_bounds[:1]
        self.lower_const_weights = []
        self.upper_const_weights = []
        self.lock_idx = None
        self.alpha_buffer_idx = 0
        # self.alpha_buffer = []


    def lock(self):
        self.lock_idx = len(self.upper_bounds)-1


    def unlock(self):
        self.lock_idx = None


    def _prepare_input(self, x: torch.Tensor) -> torch.Tensor:
        return torch.cat((x.flatten(start_dim=1), torch.ones(x.shape[0], 1)), dim=1).unsqueeze(1)


    def _get_output(self, x: torch.Tensor, shape: tuple[int, ...]) -> torch.Tensor:
        return x[:, :, :-1].view(shape)


    def _get_bounds(self, lower_bound: torch.Tensor, upper_bound: torch.Tensor, W_l: torch.Tensor, W_u: torch.Tensor, outshape: tuple[int, ...]) -> tuple[torch.Tensor, torch.Tensor]:
        lower_bound = self._prepare_input(lower_bound)
        upper_bound = self._prepare_input(upper_bound)
        W_l_pos = W_l.clamp_min(0)
        W_l_neg = W_l.clamp_max(0)
        W_u_pos = W_u.clamp_min(0)
        W_u_neg = W_u.clamp_max(0)
        return self._get_output(lower_bound @ W_l_pos + upper_bound @ W_l_neg, outshape), \
                self._get_output(upper_bound @ W_u_pos + lower_bound @ W_u_neg, outshape)


    def forward(self, model: torch.nn.Module):
        if type(model) in self.forward_net:
            self.forward_net[type(model)](model)
        else:
            raise NotImplementedError(f"{type(model)} is not supported")


    def append_constraints(self, bounds: tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]):
        l_bound, u_bound, l_const_weight, u_const_weight = bounds
        if DEBUG:
            assert not torch.isnan(l_bound).any(), "NaN detected in lower bound"
            assert not torch.isnan(u_bound).any(), "NaN detected in upper bound"
            assert not torch.isnan(l_const_weight).any(), "NaN detected in lower constraint weights"
            assert not torch.isnan(u_const_weight).any(), "NaN detected in upper constraint weights"
        if DEBUG:
            assert (l_bound > u_bound).sum() == 0
        idx = len(self.upper_bounds)-1
        if idx > 0 and self.lock_idx != idx and torch.allclose(self.lower_const_weights[-1], self.upper_const_weights[-1]):
            W_l, W_u = backsubstitute(self.lower_const_weights[-1], self.upper_const_weights[-1], l_const_weight, u_const_weight)
            new_bounds = self._get_bounds(self.lower_bounds[-2], self.upper_bounds[-2], W_l, W_u, l_bound.shape)
            self.lower_bounds[-1] = torch.max(new_bounds[0], l_bound)
            self.upper_bounds[-1] = torch.min(new_bounds[1], u_bound)
            self.lower_const_weights[-1] = W_l
            self.upper_const_weights[-1] = W_u
        else:
            self.lower_bounds.append(l_bound)
            self.upper_bounds.append(u_bound)
            self.lower_const_weights.append(l_const_weight)
            self.upper_const_weights.append(u_const_weight)
        self._upd_from_back()


    def append_and_verify(self, true_output: int) -> bool:
        last_layer = eliding_w(self.upper_bounds[-1].shape[1], true_output)
        self.append_constraints(self._affine(self.lower_bounds[-1], self.upper_bounds[-1], last_layer))
        n_batch = self.upper_bounds[-1].shape[0]
        return any([all(self.upper_bounds[-1][b] <= 0) for b in range(n_batch)])


    def _upd_from_back(self):
        W_l = self.lower_const_weights[-1]
        W_u = self.upper_const_weights[-1]
        out_shape = self.lower_bounds[-1].shape
        for i in range(len(self.upper_bounds)-3, -1, -1):
            W_l, W_u = backsubstitute(self.lower_const_weights[i], self.upper_const_weights[i], W_l, W_u)
            new_bounds = self._get_bounds(self.lower_bounds[i], self.upper_bounds[i], W_l, W_u, out_shape)
            if DEBUG:
                assert (new_bounds[0] > new_bounds[1]).sum() == 0
            if DEBUG:
                assert (self.lower_bounds[-1] > self.upper_bounds[-1]).sum() == 0, (self.lower_bounds[-1] - self.upper_bounds[-1]).max()
            self.lower_bounds[-1] = torch.max(new_bounds[0], self.lower_bounds[-1])
            self.upper_bounds[-1] = torch.min(new_bounds[1], self.upper_bounds[-1])
            if DEBUG:
                assert (self.lower_bounds[-1] > self.upper_bounds[-1]).sum() == 0, (self.lower_bounds[-1] - self.upper_bounds[-1]).max()


    def _sequential(self, net: torch.nn.Sequential) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        for i, module in enumerate(net.children()):
            if type(module) in self.forward_layer:
                self.append_constraints(self.forward_layer[type(module)](self.lower_bounds[-1], self.upper_bounds[-1], module))
                if DEBUG:
                    assert (self.lower_bounds[-1] > self.upper_bounds[-1]).sum() == 0
            else:
                self.forward_net[type(module)](module)


    def _relu(self, l_bound: torch.Tensor, u_bound: torch.Tensor, layer: torch.nn.ReLU) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        n_batch = u_bound.shape[0]
        l_flatten = l_bound.flatten(start_dim=1)
        u_flatten = u_bound.flatten(start_dim=1)
        l_new = l_bound.clamp(min=0)
        u_new = u_bound.clamp(min=0)

        above_diag = torch.ones_like(u_flatten) * (l_flatten >= 0)
        above_diag = torch.concat([above_diag, torch.zeros(n_batch, 1)], dim=1)
        u_above_weight = torch.diag_embed(above_diag)
        l_above_weight = torch.diag_embed(above_diag)

        dominator = torch.where(
            (u_flatten > 0) & (l_flatten < 0), (u_flatten - l_flatten + EPS) ** -1,
            0.0
        )
        upper_slope = u_flatten * dominator
        upper_slope = torch.concat([upper_slope, torch.zeros(n_batch, 1)], dim=1)
        u_cross_weight = torch.diag_embed(upper_slope)
        u_cross_weight[:, -1, :-1] = -l_flatten * upper_slope[:, :-1]

        l_cross_weight = torch.zeros_like(l_above_weight)
        if self.alpha_buffer_idx == len(self.alpha_buffer):
            # lower_slope = torch.nn.Parameter(INIT_COEF * torch.where(u_flatten.abs() > l_flatten.abs(), 1.0, -1.0))
            lower_slope = torch.nn.Parameter(torch.randn(u_flatten.size()) * torch.where(u_flatten.abs() > l_flatten.abs(), 1.0, -1.0))
            self.alpha_buffer.append(lower_slope)
        else:
            lower_slope = self.alpha_buffer[self.alpha_buffer_idx]

        l_cross_weight[:, :-1, :-1] = torch.diag_embed(torch.where(((u_flatten > 0) & (l_flatten < 0)), torch.sigmoid(lower_slope), 0.0))
        self.alpha_buffer_idx += 1

        l_weight = l_above_weight + l_cross_weight
        u_weight = u_above_weight + u_cross_weight
        l_weight[:, -1, -1] = u_weight[:, -1, -1] = 1
        if DEBUG:
            assert not torch.isnan(l_new).any(), "NaN detected in lower bound"
            assert not torch.isnan(u_new).any(), "NaN detected in upper bound"
            assert not torch.isnan(l_weight).any(), "NaN detected in lower constraint weights"
            assert not torch.isnan(u_weight).any(), "NaN detected in upper constraint weights"
        return l_new, u_new, l_weight, u_weight


    def _relu6(self, module: torch.nn.ReLU6) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        def six_minus(t: tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
            l_bound, u_bound = 6 - t[1], 6 - t[0]
            l_const_weight = torch.zeros_like(t[2])
            l_const_weight[:, :-1, :-1] = -t[3][:, :-1, :-1]
            l_const_weight[:, -1, :-1] = 6-t[3][:, -1, :-1]
            l_const_weight[:, -1, -1] = 1
            u_const_weight = torch.zeros_like(t[3])
            u_const_weight[:, :-1, :-1] = -t[2][:, :-1, :-1]
            u_const_weight[:, -1, :-1] = 6-t[2][:, -1, :-1]
            u_const_weight[:, -1, -1] = 1
            if DEBUG:
                assert not torch.isnan(l_bound).any(), "NaN detected in lower bound"
                assert not torch.isnan(u_bound).any(), "NaN detected in upper bound"
                assert not torch.isnan(l_const_weight).any(), "NaN detected in lower constraint weights"
                assert not torch.isnan(u_const_weight).any(), "NaN detected in upper constraint weights"
            return l_bound, u_bound, l_const_weight, u_const_weight

        self.append_constraints(six_minus(self._relu(self.lower_bounds[-1], self.upper_bounds[-1], None)))
        self.append_constraints(six_minus(self._relu(self.lower_bounds[-1], self.upper_bounds[-1], None)))


    def _conv2d(
        self, l_bound: torch.Tensor, u_bound: torch.Tensor, layer: torch.nn.Conv2d
    ) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        if DEBUG:
            assert (l_bound > u_bound).sum() == 0
        # shadow the setting and weight of conv2d layer
        w_kernel = layer.weight
        b = layer.bias
        stride = layer.stride
        padding = layer.padding
        assert layer.dilation == (1, 1), "Dilation is not supported"
        assert layer.padding_mode == "zeros", "Padding mode other than zeros is not supported"
        _, in_channel, in_height, in_width = u_bound.shape

        # Masking positive and negative weights separately
        w_kernel_postive = w_kernel.clamp(min=0)
        w_kernel_negative = w_kernel.clamp(max=0)
        if DEBUG:
            assert torch.allclose(
                F.conv2d(l_bound, w_kernel, b, stride, padding),
                layer(l_bound),
            ), (F.conv2d(l_bound, w_kernel, b, stride, padding) - layer(l_bound)).abs().max()

        # conv is basically shifting affine, therefore we can directly reuse F.conv2d with
        # bias terms are assembled in the conv2d's bias argument
        l_new = F.conv2d(
            l_bound, w_kernel_postive, b, stride, padding
        ) + F.conv2d(u_bound, w_kernel_negative, None, stride, padding)
        u_new = F.conv2d(
            u_bound, w_kernel_postive, b, stride, padding
        ) + F.conv2d(l_bound, w_kernel_negative, None, stride, padding)
        if DEBUG:
            assert (l_new > u_new).sum() == 0

        _, out_channel, out_height, out_width = u_new.shape

        # calculate the linear constraint
        conv_mat = torch.zeros(
            in_channel * in_height * in_width + 1,
            out_channel * out_height * out_width + 1,
        )
        input = torch.eye(in_channel * in_height * in_width).reshape(-1, in_channel, in_height, in_width)
        conv_mat[:-1, :-1] = layer(input).flatten(start_dim=1)
        conv_mat[-1, :-1] = b.repeat(out_height * out_width, 1).T.flatten()
        conv_mat[:-1, :-1] -= conv_mat[-1, None, :-1]
        conv_mat[-1, -1] = 1
        if DEBUG:
            input = u_bound
            output = layer(l_bound)
            output_prime = self._get_output(self._prepare_input(l_bound) @ conv_mat, output.shape)
            assert torch.allclose(output, output_prime, atol=1), (output - output_prime).abs().max()
            l_new_prime = self._get_output(self._prepare_input(l_bound) @ conv_mat.clamp_min(0) + self._prepare_input(u_bound) @ conv_mat.clamp_max(0), l_new.shape)
            u_new_prime = self._get_output(self._prepare_input(u_bound) @ conv_mat.clamp_min(0) + self._prepare_input(l_bound) @ conv_mat.clamp_max(0), u_new.shape)
            assert torch.allclose(l_new, l_new_prime, atol=1), (l_new - l_new_prime).abs().max()
            assert torch.allclose(u_new, u_new_prime, atol=1), (u_new - u_new_prime).abs().max()
        return l_new, u_new, conv_mat, conv_mat


    def _flatten(self, l_bound: torch.Tensor, u_bound: torch.Tensor, layer: torch.nn.Flatten) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        n_batch = u_bound.shape[0]
        l_new = layer(l_bound)
        u_new = layer(u_bound)
        
        l_const_weight = torch.eye(l_new.shape[1]+1).repeat(n_batch, 1, 1)
        u_const_weight = torch.eye(u_new.shape[1]+1).repeat(n_batch, 1, 1)

        return l_new, u_new, l_const_weight, u_const_weight

 
    def _skip(self, net: torch.nn.Module) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        self.lock()
        upd_from = len(self.upper_bounds)-1
        skipdim = self.upper_bounds[-1].shape[1]
        for i, module in enumerate(net.path.children()):
            if type(module) in self.forward_layer:
                self.append_constraints(self.forward_layer[type(module)](self.lower_bounds[-1], self.upper_bounds[-1], module))
            else:
                self.forward_net[type(module)](module)
        def carry(bound0: torch.Tensor, bound1: torch.Tensor, const_weights: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
            new_bound = torch.cat([bound1, bound0[:, -skipdim:]], dim=1)
            new_weights = torch.zeros(const_weights.shape[0], bound0.shape[1]+1, new_bound.shape[1]+1)
            new_weights[:, :const_weights.shape[1]-1, :const_weights.shape[2]-1] = const_weights[:, :-1, :-1]
            new_weights[:, -1, :-skipdim-1] = const_weights[:, -1, :-1]
            new_weights[:, -skipdim-1:, -skipdim-1:] += torch.eye(skipdim+1)
            return new_bound, new_weights
        for i in range(upd_from+1, len(self.upper_bounds)):
            self.lower_bounds[i], self.lower_const_weights[i-1] = carry(self.lower_bounds[i-1], self.lower_bounds[i], self.lower_const_weights[i-1])
            self.upper_bounds[i], self.upper_const_weights[i-1] = carry(self.upper_bounds[i-1], self.upper_bounds[i], self.upper_const_weights[i-1])
        self.unlock()
        self.append_constraints(self._affine(self.lower_bounds[-1], self.upper_bounds[-1], skip_application(skipdim)))

 
    def _linear(self, l_bound: torch.Tensor, u_bound: torch.Tensor, layer: torch.nn.Linear) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        return self._affine(l_bound, u_bound, layer.weight, layer.bias)
    

    def _affine(self, l_bound: torch.Tensor, u_bound: torch.Tensor, w: torch.nn.Module, b: Optional[torch.nn.Module] = None) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        n_batch = u_bound.shape[0]
        w = w.T

        w_postive = w.clamp(min=0)
        w_negative = w.clamp(max=0)

        l_new = l_bound @ w_postive + u_bound @ w_negative
        u_new = u_bound @ w_postive + l_bound @ w_negative
        const_weight = torch.zeros(n_batch, w.shape[0]+1, w.shape[1]+1)
        const_weight[:, :-1, :-1] = w
        const_weight[:, -1, -1] = 1

        if b is not None:
            const_weight[:, -1, :-1] = b
            l_new += b
            u_new += b

        return l_new, u_new, const_weight, const_weight

**can modify dp_verify here**

In [ ]:
VERBOSE = True
def dp_verify(model: torch.nn.Module, input: torch.Tensor, eps: float, true_output: int) -> bool:
    input = input.unsqueeze(0)
    shallowuni = Deeppoly(input-eps, input+eps)
    shallowuni.forward(model)
    if shallowuni.append_and_verify(true_output):
        return True
    if shallowuni.alpha_buffer_idx == 0:
        return False
    for i in range(shallowuni.alpha_buffer_idx):
        shallowuni.alpha_buffer[i].requires_grad = True

    if DEBUG:
        torch.autograd.set_detect_anomaly(True)

    optimizer = torch.optim.Adam(shallowuni.alpha_buffer, lr=LR)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=DECAY_STEP, gamma=DECAY)
    for epoch in range(EPOCHS):
        optimizer.zero_grad()
        shallowuni.rewind()
        shallowuni.forward(model)
        if shallowuni.append_and_verify(true_output):
            return True
        loss = torch.mean(torch.relu(shallowuni.upper_bounds[-1]).flatten())
        loss.backward()
        optimizer.step()
        if VERBOSE:
            print(f"Epoch {epoch+1}, Loss: {loss.item()}")
            print(sum(shallowuni.alpha_buffer[-1].grad))
        if scheduler.get_last_lr()[0] > 0.1:
            scheduler.step()
    return False


### Case Recording

**| model | testing_case | ground_truth || testing_result | others**

- conv_linear | img_mnist_0.072113.txt | not verified ||
- conv_linear | img_mnist_0.079101.txt | verified     ||
- fc_linear | img_mnist_0.077340.txt | not verified   ||
- fc_linear | img_mnist_0.082864.txt | verified       ||
- fc6_d | img_mnist_0.089335.txt | not verified       ||
- fc6_d | img_mnist_0.050621.txt | verified           ||
- fc6_w | img_mnist_0.124546.txt | not verified       ||
- fc6_w | img_mnist_0.042779.txt | verified           ||
- fc_dw | img_mnist_0.071216.txt | not verified       ||
- fc_dw | img_mnist_0.030104.txt | verified           ||
- fc6_base | img_mnist_0.074530.txt | not verified    ||
- fc6_base | img_mnist_0.044957.txt | verified        ||
- fc_base | img_mnist_0.048508.txt | not verified     ||    not | no gradient problem
- fc_base | img_mnist_0.048839.txt | verified         ||    not | no gradient problem
- conv6_base | img_cifar10_0.003450.txt | not verified||
- conv6_base | img_cifar10_0.003959.txt | verified    ||
- fc6_dw | img_mnist_0.082344.txt | not verified      ||
- fc6_dw | img_mnist_0.034540.txt | verified          ||
- fc_d | img_mnist_0.059808.txt | not verified        ||
- fc_d | img_mnist_0.040164.txt | verified            ||
- fc_w | img_mnist_0.110494.txt | not verified        ||
- fc_w | img_mnist_0.042224.txt | verified            ||
- conv_d | img_mnist_0.103841.txt | not verified      ||    
- conv_d | img_mnist_0.078071.txt | verified          ||    
- conv_base | img_mnist_0.075798.txt | not verified   ||    not verified (√)
- conv_base | img_mnist_0.055929.txt | verified       ||    verified (√)
- skip6 | img_mnist_0.102399.txt | not verified       ||
- skip6 | img_mnist_0.074744.txt | verified           ||
- skip_large | img_mnist_0.131450.txt | not verified  ||
- skip_large | img_mnist_0.041155.txt | verified      ||
- skip | img_mnist_0.139199.txt | not verified        ||  
- skip | img_mnist_0.078863.txt | verified            ||
- skip6_large | img_mnist_0.068828.txt | not verified ||
- skip6_large | img_mnist_0.019448.txt | verified     ||

In [5]:
import torch
from utils.loading import parse_spec
from networks import get_network

model = "fc_base"
dataset = "mnist"

if dataset == "mnist":
    in_ch, in_dim, num_class = 1, 28, 10
elif dataset == "cifar10":
    in_ch, in_dim, num_class = 3, 32, 10
else:
    raise ValueError(f"Unknown dataset: {dataset}")

resuls = {}
for _, _, files in os.walk(f"C:/Users/TTTeq/Documents/CodeFolder/CP/VCS/RTAI/rtai-project-21/test_cases/{model}/"):
    for file in files:
        file_path = os.path.join(f"C:/Users/TTTeq/Documents/CodeFolder/CP/VCS/RTAI/rtai-project-21/test_cases/{model}/", file)
        true_label, dataset, image, eps = parse_spec(file_path)
        print(file_path.split("/")[-1])
        net = get_network(
            model,
            in_ch=in_ch,
            in_dim=in_dim,
            num_class=num_class,
            weight_path=f"models/{dataset}_{model}.pt",
        ).to("cpu")
        
        print(net)
        resuls[f"{model}/{file}"] = dp_verify(net, image, eps, true_label)
print(resuls)

img_mnist_0.048508.txt
Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=50, bias=True)
  (2): ReLU()
  (3): Linear(in_features=50, out_features=50, bias=True)
  (4): ReLU()
  (5): Linear(in_features=50, out_features=50, bias=True)
  (6): ReLU()
  (7): Linear(in_features=50, out_features=10, bias=True)
)
Epoch 1, Loss: 18.246410369873047
tensor([ 0.0000e+00, -7.4184e-03,  0.0000e+00,  0.0000e+00,  4.3374e-03,
        -2.3402e-02,  3.5677e-04,  0.0000e+00,  2.8486e-02,  0.0000e+00,
         0.0000e+00, -6.3798e-02,  4.0010e-01, -2.0677e-02, -1.3955e-01,
        -3.0210e-02, -4.2865e-02,  7.6041e-03, -4.6965e-02,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00, -2.1901e-03, -2.0857e-02,
         1.5447e-02,  4.6962e-04, -9.5991e-03,  1.7386e-02, -5.2021e-02,
        -2.8613e-02,  1.6409e-02,  0.0000e+00, -7.9364e-03, -5.4014e-02,
         5.6496e-04, -2.2199e-03,  0.0000e+00, -2.1288e-01,  0.0000e+00,
         1.3848e-02,  2.9042e-